# Using eigenvectors to find clusters in Graphs

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import networkx as nx

# ZACHARY KARATE CLUB

dataset [zachary](http://vlado.fmf.uni-lj.si/pub/networks/data/Ucinet/UciData.htm#zachary)

The graph is based on a real-life social experiment at a university karate club. Zachary[1] observed the interactions outside of the dojo between 34 members of the club over a period of three years. During his study, a conflict arose between the administrator of the club, referred to as "Mr. Hi", and the club's instructor, often labeled as "John A", which eventually led to the club splitting into two different clubs, each led by one of these two individuals.

When you run community detection algorithms, such as spectral clustering, on this graph, they should ideally detect the two communities corresponding to the split in the karate club.

1: Zachary W. (1977). An information flow model for conflict and fission in small groups. Journal of Anthropological Research, 33, 452-473.

In [ ]:
G = nx.karate_club_graph()
pos = nx.spring_layout(G, seed=46)
nx.draw(G, with_labels=True, pos=pos)

In [ ]:
# Create a dictionary to map the labels to numbers for coloring
label_map = {'Mr. Hi': 0, 'Officer': 1}

# Extract the labels and convert them to numbers
node_colors = [label_map[G.nodes[node]['club']] for node in G.nodes()]
nx.draw(G, node_color=node_colors, with_labels=True, pos=pos)

Imagine you have a network of friends where the nodes are you and your friends, and the lines (edges) between the nodes are the friendships. The Laplacian matrix is like a ledger that helps keep track of the friendships in a very particular way.

Here's how you make this ledger:

- Who has how many friends (number of edges from every node): First, you write down a list of how many friends each person has. This is like a tally for each person.
- Who is friends with whom (edges between nodes): Next, you make a grid where you mark who is directly friends with whom.

More technically, this is defined like this:
- Degree Matrix (D): This is a diagonal matrix where each diagonal element D[i][i] equals the sum of the weights of the edges connected to node i. For an unweighted graph, it's just the number of edges connected to node i.
- Adjacency Matrix (A): This is a square matrix that represents the graph. If there is an edge between nodes i and j, then A[i][j] is positive (often 1 for unweighted graphs, or a weight for weighted graphs). If there is no edge, A[i][j] is zero.

Now, the Laplacian matrix (L) combines this information by doing a simple calculation: for each person, subtract the number of direct friends they have from the number of total friends.

 $$L = D - A$$

This matrix has some neat properties:

If you add up all the numbers in any row or column, the sum is zero. It's like saying, "This person has as many friends as they have friends."

The way the numbers are arranged and the values they have can tell you if there are groups of friends who are all connected to each other but not much to others in the network. It's like spotting cliques in your group.

The way the numbers in the Laplacian matrix are laid out can reveal hidden structures in the network, such as which groups of friends might form a club together or how you could split the group into two or more separate parties. It's a mathematical way to explore the social circles within the network.

When we cluster a network, we can use eigenvectors to help us see the shape of the network and the underlying structure. The eigenvectors reveal the most important connections and ignore the less important ones. 
They reveal clusters within the graph, as nodes within the same cluster tend to be represented close to each other in the low-dimensional space created by a subset of the eigenvectors.

In [ ]:
# Create the Adjacency Matrix
A = nx.adjacency_matrix(G).todense()

# Create the Degree Matrix
D = np.diag(A.sum(axis=1).flatten())

# Compute the Laplacian Matrix
L = D - A

In the context of Principal Component Analysis (PCA) and other variance-based methods, larger eigenvalues indeed correspond to greater variance in the data, and thus the eigenvectors associated with the largest eigenvalues are typically the most significant. However, when it comes to the Laplacian matrix of a graph and spectral clustering, the smallest non-zero eigenvalues have a different and special role.

The reason we look at the smallest non-zero eigenvalues (also known as the Fiedler value in the case of the second-smallest eigenvalue) when using the Laplacian matrix for clustering is due to the properties of the Laplacian and what these eigenvalues represent in the context of graph connectivity:


- The smallest eigenvalue is always zero, and there is always at least one eigenvector associated with this zero eigenvalue. If there is exactly one eigenvalue that is zero (with its corresponding eigenvector), it indicates that the graph is connected; there is a path between every pair of nodes. If there are multiple eigenvalues that are zero (each with its corresponding eigenvector), the number of such zero eigenvalues indicates the number of disconnected components in the graph. Each of these eigenvectors will be constant within its component and zero elsewhere.

- The second-smallest eigenvalue, or the first non-zero eigenvalue, provides insight into how the graph is connected. It helps to identify a cut that can partition the graph into two sparsely connected subgraphs. In a way, this value measures how easy it is to "cut" the graph into separate pieces.

It can make sense to use more than the first two eigenvectors for clustering larger graphs, especially when the structure of the graph is complex and there are multiple clusters to identify.

In [ ]:
# Compute the eigenvalues and eigenvectors
vals, vecs = np.linalg.eigh(L)

# Sort these based on the eigenvalues from smallest to largest
vecs = vecs[:,np.argsort(vals)]
vals = vals[np.argsort(vals)]

In [ ]:
# we have 34 eigenvectors
vecs.shape

In [ ]:
# and 34 eigenvalues
plt.scatter(np.arange(len(vals)), vals)

Now lets visualize the eigenvectors. We will create two types of visualisation: 
One where the nodes are ordered on the x-axis based on there node-number (note that this will create some structure, which might be misleading) and on the y-axis the values of the eigenvectors are plotted.

For the second eigenvector you can see that splitting the data around 0.0 will nicely split the data into two clusters that corresponds with the original labels!

In [ ]:
n = 4
fig, ax = plt.subplots(2, n, figsize=(12, 8))

for i in range(n):
    v = vecs[:,i]
    sns.scatterplot(x=range(len(v)), y=v, hue=node_colors, ax=ax[0, i], legend=False)
    sns.histplot(x=v, ax=ax[1, i], bins=10, hue=node_colors, legend=False, kde=True)
    ax[0, i].set_title(f'Eigenvector {i+1}')
    ax[0, i].set_xlabel('Node number')
    ax[1, i].set_xlabel('Bin')

How would you do this if it is more complex? You can run a sklearn clustering algorithm like KMeans (we will explore clustering models later on).

In [ ]:
v = vecs[:, 1].reshape(-1, 1)

kmeans = KMeans(n_clusters=2)
clusters = kmeans.fit_predict(v)

Let's use the clusters in the eigenvector to color the nodes and lets see how well we did!

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
# Use a color map
color_map = plt.cm.get_cmap('viridis', 2)

# Draw the nodes - the node color varies with the cluster
nx.draw(G, pos, node_color=[color_map.colors[i] for i in clusters], with_labels=True, node_size=300, edge_color='gray', ax=ax[0])
nx.draw(G, node_color=node_colors, with_labels=True, pos=pos, ax=ax[1])

ax[0].set_title('Clusters from Eigenvectors')
ax[1].set_title('True Club Membership')
